# Правильная цена

Сегодня мы создаем более сложное решение для оценки цен на товары.

1. Этот блокнот: создаем базу данных RAG с нашими 400 000 обучающими данными
2. Блокнот дня 2.1: визуализируем в 2D
3. День 2.2 записная книжка: визуализация в 3D
4. День 2.3 записная книжка: создание и тестирование конвейера RAG с помощью GPT-4o-mini
5. День 2.4 Записная книжка: (а) вернем нашу случайную цену на лес (б) Создадим общую цену, в которую смогут внести свой вклад все участники закупки

Уф! Столько всего нужно проработать за один день!

## ПОЖАЛУЙСТА, ОБРАТИТЕ ВНИМАНИЕ:

У нас уже есть очень мощный инструмент оценки продукта с нашим фирменным, тщательно настроенным LLM. Большинство людей были бы очень довольны этим! Основная причина, по которой мы добавляем эти дополнительные шаги, заключается в том, чтобы углубить ваши знания в области RAG и рабочих процессов Agentic.


In [ ]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

## Вернемся к файлам pkl

Как бы нам ни понравилась работа с данными на шестой неделе, мы, вероятно, не хотим проходить весь этот процесс снова!

Давайте повторно используем файлы pkl, которые мы создали тогда. Либо скопируйте файлы "train.pkl" и "test.pci" из папки "Неделя 6" в папку "Неделя 8", либо вы также можете загрузить их отсюда:

https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW?usp=drive_link

In [ ]:
# With train.pkl in this folder, you can run this:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [ ]:
train[0].prompt

# Теперь создайте хранилище цветовых данных

На пятой неделе мы создали хранилище цветовых данных со 123 документами, представляющими фрагменты объектов нашей вымышленной компании Insurellm.

Теперь мы создадим хранилище данных Chroma с 400 000 продуктов из нашего обучающего набора данных! Это становится реальностью!

Обратите внимание, что мы не будем использовать длинную цепочку, но API очень прост и соответствует предыдущему.

Особое примечание: если Chrome выходит из строя, а вы являетесь пользователем Windows, вам следует попробовать вернуться к более ранней версии библиотеки Chroma с помощью:  
`!pip install chromadb==0.5.0`  
Большое спасибо студентке Келли З. за то, что она выяснила это и указала на проблему с GitHub [здесь](https://github.com/chroma-core/chroma/issues/2513). 
Но сначала попробуйте не возвращать цветность.

In [ ]:
client = chromadb.PersistentClient(path=DB)

In [ ]:
# Check if the collection exists and delete it if it does
collection_name = "products"

# For old versions of Chroma, use this line instead of the subsequent one
# existing_collection_names = [collection.name for collection in client.list_collections()]
existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

# Представляем трансформатор предложений

Полностью мини-LM - это очень полезная модель от Hugging Face, которая отображает предложения и абзацы в 384-мерном плотном векторном пространстве и идеально подходит для таких задач, как семантический поиск.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Ее можно довольно быстро запустить локально.

В прошлый раз мы использовали OpenAI embeddings для создания векторных вложений. Преимущества по сравнению с OpenAI embeddings:
1. Это бесплатно и быстро!
3. Мы можем запускать это локально, поэтому данные никогда не покидают наш ящик - может пригодиться, если вы создаете персональный RAG


In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]
vector

In [ ]:
# Quick sidebar - extra to the videos - a function to compare vectors

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [ ]:
# And let's see how adding a few words to the context can change things up!

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

In [ ]:
# OK back to the main story - let's make something we can vectorize

def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [ ]:
description(train[0])

## Теперь мы заполняем наше хранилище данных RAG

В следующей ячейке находятся 400 000 элементов в Chrome.

Не стесняйтесь сокращать количество документов, если это займет слишком много времени! Вы можете изменить на:  
`NUMBER_OF_DOCUMENTS = 20000`  
И этого достаточно для совершенно хорошего конвейера RAG.

Просто обратите внимание, что если вы прервете работу приведенной ниже ячейки во время ее выполнения, вам, возможно, потребуется очистить хранилище данных Chroma (повторно запустив предыдущую ячейку, которая удалила коллекцию), прежде чем запускать ее снова. В противном случае программа сообщит, что существуют документы с таким же идентификатором.

In [ ]:
NUMBER_OF_DOCUMENTS = len(train)

# Uncomment if you'd rather not wait for the full 400,000
# NUMBER_OF_DOCUMENTS = 20000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )